## Wordcount

La idea de este ejemplos es obtener la cantidad de apariciones de cada palabra en todas las obras de shakespeare.
Fuente de Datos: http://www.gutenberg.org/cache/epub/100/pg100.txt (se elimino la introduccion y licencia).


In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [ ]:
shakespeareRDD = sc.textFile('data/shakespeare.txt',8)

In [ ]:
shakespeareRDD.take(20)

In [ ]:
wordsRDD = shakespeareRDD.flatMap(lambda line: line.split())
wordsRDD.take(10)

In [ ]:
wordsCountRDD = wordsRDD.map(lambda word: (word,(1,2)))
wordsCountRDD.take(5)

In [ ]:
wordsCountRDD.reduceByKey(lambda a,b: (a[0]+b[0],1)).sortBy(ascending=False,keyfunc=lambda x:x[1][0]).take(10)

## N-Grams

### Que es un n-grama

Es una secuencia continua de n caracteres de una cadena de texto.

In [ ]:
def trigrams(t):
    t=t.lower()
    return [t[i:i+3] for i in range(0, len(t) - 2)]

In [ ]:
trigrams("hola datos")

## Indiana Jones: En busca de todos los trigramas de todas las obras de shakespeare

In [ ]:
anotherShakespeareRDD = sc.textFile('data/shakespeare.txt',8)

In [ ]:
trigramsRDD = anotherShakespeareRDD.flatMap(trigrams).filter(lambda a : a != '   ')

In [ ]:
trigramsRDD.take(10)

### Calculando la frecuencia de cada trigrama

In [ ]:
trigramsCount = trigramsRDD.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
print(trigramsCount.take(5))

In [ ]:
trigramsCountSorted = trigramsCount.sortBy(ascending=False,keyfunc=lambda x:x[1])
print(trigramsCountSorted.take(20))

In [ ]:
trigramsCountSorted.take(10)

### Caculando la frecuencia total de la coleccion

In [ ]:
totalFrec2 = trigramsCountSorted.map(lambda x: x[1]).reduce(lambda x,y: x+y)
totalFrec1 = trigramsRDD.count()

In [ ]:
assert totalFrec2 == totalFrec1

In [ ]:
totalFrec1

In [ ]:
totalFrec2

In [ ]:
print(totalFrec1)

In [ ]:
print(trigramsRDD.count())

### Calculando la probabilidad de cada trigrama

In [ ]:
#print(trigramsCountSorted.take(5))
trigramsProb = trigramsCountSorted.map(lambda x: (x[0],round(float(x[1])/totalFrec1,3)))

In [ ]:
trigramsProb.take(10)